In [5]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model

# Load the dataset
df = pd.read_csv("C:/Users/Jabasingh Daniel/Downloads/cobol_dataset2.csv")

In [6]:
tokenizer_cobol = Tokenizer(filters='')
tokenizer_cobol.fit_on_texts(df['cobol code'])
tokenizer_summary = Tokenizer(filters='')
tokenizer_summary.fit_on_texts(df['summary'])

# Define the vocabulary sizes
vocab_size_cobol = len(tokenizer_cobol.word_index) + 1
vocab_size_summary = len(tokenizer_summary.word_index) + 1

In [7]:
#tokenizer_cobol.index_word

In [9]:
#tokenizer_summary.index_word

In [11]:
# Prepare input-output pairs
X = tokenizer_cobol.texts_to_sequences(df['cobol code'])
Y = tokenizer_summary.texts_to_sequences(df['summary'])
# Pad sequences
max_len_cobol = max([len(seq) for seq in X])
max_len_summary = max([len(seq) for seq in Y])

X = pad_sequences(X, maxlen=max_len_cobol, padding='post')
Y = pad_sequences(Y, maxlen=max_len_summary, padding='post')

In [12]:
# Shift the target sequences for training
Y_input = Y[:, :-1]
Y_output = Y[:, 1:]

# Define the Seq2Seq model
latent_dim = 512

encoder_inputs = Input(shape=(max_len_cobol,))
encoder_embedding = Embedding(vocab_size_cobol, latent_dim, mask_zero=True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(max_len_summary-1,))
decoder_embedding = Embedding(vocab_size_summary, latent_dim, mask_zero=True)(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(vocab_size_summary, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [14]:
# Compile and train the model
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy',metrics=['accuracy'])
model.fit([X, Y[:,:-1]], Y.reshape(Y.shape[0], Y.shape[1], 1)[:,1:],batch_size=20, epochs=100, validation_split=0.2) 


Epoch 1/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 15s 982ms/step - accuracy: 0.8172 - loss: 4.1830 - val_accuracy: 0.8501 - val_loss: 5.0072
Epoch 2/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 9s 781ms/step - accuracy: 0.8323 - loss: 3.9890 - val_accuracy: 0.8554 - val_loss: 5.0042
Epoch 3/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 7s 825ms/step - accuracy: 0.7726 - loss: 3.8915 - val_accuracy: 0.8566 - val_loss: 4.9690
Epoch 4/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 8s 839ms/step - accuracy: 0.8280 - loss: 3.7698 - val_accuracy: 0.3632 - val_loss: 4.9782
Epoch 5/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 8s 900ms/step - accuracy: 0.6177 - loss: 3.6225 - val_accuracy: 0.2505 - val_loss: 4.8996
Epoch 6/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 10s 797ms/step - accuracy: 0.4522 - loss: 3.5149 - val_accuracy: 0.2432 - val_loss: 4.9655
Epoch 7/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 10s 806ms/step - accuracy: 0.2899 - loss: 3.4606 - val_accuracy: 0.3164 - val_loss: 4.8946
Epoch 8/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 7s 801ms/step - accuracy: 0.3543 - loss: 3.4002 - val_accuracy: 0.1976 

In [15]:
# Save the model
model.save('seq2seq_model_D2.keras') 

In [18]:
#testing

import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model

# Assuming your test data is loaded and preprocessed similarly to the training data
df_test = pd.read_csv("C:/Users/Jabasingh Daniel/Downloads/cobol_dataset2.csv")
X_test = tokenizer_cobol.texts_to_sequences(df_test['cobol code'])
Y_test = tokenizer_summary.texts_to_sequences(df_test['summary'])

X_test = pad_sequences(X_test, maxlen=max_len_cobol, padding='post')
Y_test = pad_sequences(Y_test, maxlen=max_len_summary, padding='post')

# Load the trained model
model = load_model("C:/Users/Jabasingh Daniel/Desktop/EGDK/modeling/seq2seq_model_D2.keras")

# Function to calculate sequence accuracy
def sequence_accuracy(y_true, y_pred):
    correct = 0
    total = 0
    for true_seq, pred_seq in zip(y_true, y_pred):
        true_seq = true_seq[true_seq != 0]  # Remove padding
        pred_seq = pred_seq[:len(true_seq)]  # Truncate to the length of the true sequence
        if np.array_equal(true_seq, pred_seq):
            correct += 1
        total += 1
    return correct / total

# Generate predictions
Y_pred = model.predict([X_test, Y_test[:, :-1]])

# Convert predictions to sequences of token ids
Y_pred_sequences = np.argmax(Y_pred, axis=-1)

# Calculate the accuracy
accuracy = sequence_accuracy(Y_test[:, 1:], Y_pred_sequences)
print(f'Sequence Accuracy: {accuracy * 100:.2f}%')


7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 329ms/step
Sequence Accuracy: 53.85%


In [22]:
from tensorflow.keras.models import load_model

# Load the trained model
model = load_model("C:/Users/Jabasingh Daniel/Desktop/EGDK/modeling/seq2seq_model_D2.keras")

def preprocess_input(cobol_code, tokenizer_cobol, max_len_cobol):
    # Tokenize the input COBOL code
    sequence = tokenizer_cobol.texts_to_sequences([cobol_code])
    # Pad the sequence
    padded_sequence = pad_sequences(sequence, maxlen=max_len_cobol, padding='post')
    return padded_sequence


In [23]:
def predict_summary(cobol_code, model, tokenizer_cobol, tokenizer_summary, max_len_cobol, max_len_summary):
    # Preprocess the input COBOL code
    input_seq = preprocess_input(cobol_code, tokenizer_cobol, max_len_cobol)
    
    # Initialize the decoder input
    decoder_input = np.zeros((1, max_len_summary - 1))
    
    # Predict the summary
    for i in range(max_len_summary - 1):
        output_tokens = model.predict([input_seq, decoder_input])
        sampled_token_index = np.argmax(output_tokens[0, i, :])
        decoder_input[0, i] = sampled_token_index
        
        # Stop if a zero token (padding) is predicted, indicating no more meaningful tokens
        if sampled_token_index == 0:
            break
    
    # Convert token indices back to words
    predicted_summary = []
    for token in decoder_input[0]:
        if token == 0:
            continue
        word = tokenizer_summary.index_word.get(token, '')
        predicted_summary.append(word)
    
    return ' '.join(predicted_summary)


In [29]:
# Define your COBOL code to be summarized
cobol_code_example ="01 {TOTAL} PIC 9(5) VALUE 0. 01 {PRICE} PIC 9(3) VALUE 100. "





# Predict the summary
summary = predict_summary(cobol_code_example, model, tokenizer_cobol, tokenizer_summary, max_len_cobol, max_len_summary)
print(f'Summary: {summary}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
Summary: {price} to
